In [1]:
import pandas as pd
import sqlite3

In [2]:
def max_value(row):
    print(row)
    return max(row['ga_gt'], row['client_total'])

In [3]:
# Define DB
conn = sqlite3.connect(r'C:\Users\bbrown\Documents\python_scripts\db-build-trial\db_trial_v1\ga_harvest.db')
c = conn.cursor()

In [4]:
# Create a mock-up GIS dataset
data = {"INTAKE_ID": "BBB241127",
        "UNIQUE_ID": "BBB241127TH04",
        "STATE": "GA",
        "COUNTY": "Heard",
        "COMP": "122-1",
        "TRACT": "Bily",
        "STAND": "1",
        "CLIENT_ACR": 43,
        "GA_ACRES": 50,
        "EST_AGE": 23,
        "SPECIES": "Loblolly Pine",
        "ORIGIN": "Planted",
        "PRE_BA": 100,
        "PRE_TPA": 120,
        "PRE_GTA": 112,
        "PLN_BA": 50,
        "PLN_TPA": 50,
        "PLN_GTA": 50,
        "GT_CURRENT": 400,
        "GT_FUTURE": 0,
        "RP_REQUEST": 9,
        "COMMENTS": None
        }

df = pd.DataFrame(data, index=[0])

In [5]:
df

,INTAKE_ID,UNIQUE_ID,STATE,COUNTY,COMP,TRACT,STAND,CLIENT_ACR,GA_ACRES,EST_AGE,...,PRE_BA,PRE_TPA,PRE_GTA,PLN_BA,PLN_TPA,PLN_GTA,GT_CURRENT,GT_FUTURE,RP_REQUEST,COMMENTS
0,BBB241127,BBB241127TH04,GA,Heard,122-1,Bily,1,43,50,23,...,100,120,112,50,50,50,400,0,9,None


In [6]:
client_df = df[
    ['UNIQUE_ID',
     'TRACT',
     'COMP',
     'STAND',
     'CLIENT_ACR',
     'EST_AGE',
     'COUNTY',
     'STATE',
     'SPECIES',
     'ORIGIN',
     'PRE_BA',
     'PRE_TPA',
     'PRE_GTA',
     'PLN_BA',
     'PLN_TPA',
     'PLN_GTA',
     'GT_CURRENT',
     'GT_FUTURE',
     'RP_REQUEST'
     ]
].copy()

client_df.rename(columns={
    'UNIQUE_ID': 'client_uid',
    'COMP': 'cmp',
    'CLIENT_ACR': 'acres',
    'EST_AGE': 'age',
    'PRE_BA': "preba",
    "PRE_TPA": "pretpa",
    "PRE_GTA": 'pregta',
    'PLN_BA': 'plnba',
    'PLN_TPA': 'plntpa',
    'PLN_GTA': 'plngta',
    'GT_CURRENT': 'client_gt',
    'GT_FUTURE': 'future_gt',
    'RP_REQUEST': 'reporting_period'
}, inplace=True
)

client_df.columns = [x.lower().replace(' ', '_') for x in client_df.columns]

In [7]:
# normalize the species, origin, tract, county, and state


# first pull the associated tables from the DB
for table in ['species', 'origin', 'tract', 'county', 'state']:
    mapping_df = pd.read_sql(f"SELECT * from {table}", conn)
    mapping_map = dict(zip(mapping_df.iloc[:, 0], mapping_df.iloc[:, 1]))
    
    # Check to make sure the value in client_df is appropriate (should be string for all of these)
    client_df[f'{table}'] = client_df[f'{table}'].astype(str)

    # lowercase the value from the client_df
    client_df[f'{table}'] = client_df[f'{table}'].str.lower()
    
    # if client_df value not in table: update the sql table to include it
    missing_values = client_df[~client_df[f'{table}'].isin(mapping_df[f'{table}'])]
    if not missing_values.empty:
        print(f"New values in table: {table}")
        new_values = client_df[~client_df[f'{table}'].isin(mapping_df)]
        new_values = new_values[[f"{table}"]].drop_duplicates()
        max_uid = mapping_df[f'{table}_uid'].max()
        new_values[f'{table}_uid'] = range(max_uid + 1, max_uid + 1 + (len(new_values)))
        new_values.to_sql(f'{table}', conn, if_exists='append', index=False)
        mapping_df = pd.concat([mapping_df, new_values])
        mapping_map = dict(zip(mapping_df.iloc[:, 0], mapping_df.iloc[:, 1]))
        
        
    # map the values to the table    
    client_df[f"{table}"] = client_df[f"{table}"].map(mapping_map)


client_df.to_sql('client_intake', conn, if_exists='append', index=False)
    
    




1

### Work on the GA intake table

In [8]:
ga_df = df[
    [
        "UNIQUE_ID",
        "INTAKE_ID",
        "GA_ACRES",
        "PRE_GTA",
        "PLN_GTA",
        'CLIENT_ACR'
    ]
].copy()

ga_df['ga_gt'] = ga_df['GA_ACRES'] * (ga_df['PRE_GTA'] - ga_df['PLN_GTA'])
ga_df['client_total'] = ga_df['CLIENT_ACR'] * (ga_df['PRE_GTA'] - ga_df['PLN_GTA'])

ga_df.rename(columns={
    'UNIQUE_ID': 'ga_uid',
    'INTAKE_ID': 'intakeid',
    'GA_ACRES': 'ga_acres'
}, inplace=True
             )

ga_df['folderid'] = ga_df['intakeid']
ga_df['total_gt'] = ga_df.apply(lambda x: max_value(x), axis=1)

ga_df = ga_df[
    [
        'ga_uid',
        'intakeid',
        'folderid',
        'ga_acres',
        'ga_gt',
        'total_gt'
        ]
    ]

ga_df.to_sql('ga_intake', conn, if_exists='append', index=False)

ga_uid          BBB241127TH04
intakeid            BBB241127
ga_acres                   50
PRE_GTA                   112
PLN_GTA                    50
CLIENT_ACR                 43
ga_gt                    3100
client_total             2666
folderid            BBB241127
Name: 0, dtype: object


1

### Populate the expected gt by reporting period

This may have to happen before the mapping of client_df

This will be trickier

Steps:

- Discover the TCS for each activity
- See if it already exists in the TCS lookup
  - If it does add the "RPs" volume to the volume already in the table listed for that RP
  - If it does not then create a new entry
  - If there is future GT create an entry for RP + 1 and put the future gt there.


In [9]:
def tcser(row):
    if not row['cmp'] == None:
        return str(row['tract']) + "_" + str(row['cmp']) + "_" + str(row['stand'])
    else:
        return str(row['tract']) + "_" + str(row['stand'])

In [16]:
expected_df = client_df[['client_uid', 'tract', 'cmp', 'stand', 'client_gt', 'future_gt', 'reporting_period']].copy()


expected_df['tcs'] = client_df.apply(lambda x: tcser(x), axis=1)

In [17]:
client_df

,client_uid,tract,cmp,stand,acres,age,county,state,species,origin,preba,pretpa,pregta,plnba,plntpa,plngta,client_gt,future_gt,reporting_period
0,BBB241127TH04,122,122-1,1,43,23,4,0,1,0,100,120,112,50,50,50,400,0,9


In [18]:
expected_df

,client_uid,tract,cmp,stand,client_gt,future_gt,reporting_period,tcs
0,BBB241127TH04,122,122-1,1,400,0,9,122_122-1_1


In [13]:
# Now look up that tcs and see if it exists

### Uff. I don't have this table made yet... I thought I did.

tcs_table = pd.read_sql('Select * FROM "tcs"', conn)

In [14]:
tcs_table.iloc[0]

tcs_uid                 0
ticketid    BJT010101TH02
tcs                913_10
Name: 0, dtype: object

In [15]:
pd.merge(expected_df, tcs_table, how='left', on='tcs')

,client_uid,tract,cmp,stand,client_gt,future_gt,reporting_period,tcs,tcs_uid,ticketid
0,BBB241127TH04,122,122-1,1,400,0,9,122_122-1_1,NaN,NaN


Ok. If TCS / TIcketID is new : create the new TicketID entry to:
-  'ticketid'
-  'ticektid_lookup_activity_id'
-  and 'tcs'
  
  ALSO:
  
  Make a new entry to the gt_expected table


  IF the TCS / TicketID already exists:
  - NOT SURE!!!
  - It will be pending, so I am not sure if we will want to add the volume to the gt_expected table.
  - This is hard because the intake is a smaller unit, but as time goes on the subtraction from mill tickets is at a larger unit

## Thoughts

- I want to make sure the data going into the database is not already there
- I want to make sure that the data in the data frame is of the correct type
- I want to make sure that the data in the database is protected and not goign to get lost.

In [ ]:
client_df.to_sql('client_intake', conn, if_exists='append', index=False)